In [ ]:
import os
import time
import random
import requests
import pandas as pnd
import BggHelper as BH
import BggDbGetter as BDG
import BggDbScraper as BDS
from langdetect import detect
from tqdm.notebook import tqdm
from boardgamegeek import BoardGameGeek

# Step 1

Access BBG API's to download game and user preferences data.
Define criterion used for setting the parameters that drive the download in termis of the variables defining the dataset, such collecting games only for a specified period or a specified set of categories. The wider the game collection, the better.

In [ ]:
helper = BH.BggHelper()
helper.bgg_start(get=False, scrape=False)

#### Dataframe reading

In [13]:
df = pnd.read_csv("BGG_sampled.csv")
print(df.columns)

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'families', 'categories',
       'mechanics', 'designers', 'artists', 'publishers', 'description',
       'yearpublished', 'minplayers', 'maxplayers', 'playingtime', 'minage',
       'name', 'usersrated', 'average', 'bayesaverage', 'stddev', 'median',
       'owned', 'numweights', 'averageweight', 'ranks'],
      dtype='object')


# Step 2
Pre-process game text (title and description) in order to define the terms to include in the variables to be observed, eventually using a word embedding model to normalize and/or expand the terminology extracted from the game description.

# TODO: Make class to clean dataframe and keep only selected language (English) descriptions

Should be something like below

In [ ]:
langs = list()
for row in tqdm(list(df.iterrows())):
    try:
        langs.append(detect(row[1]["description"]))
    except:
        pass

In [ ]:
set_langs = set(langs)
for i in set_langs:
    print("{0}: {1}".format(i, langs.count(i)))

In [16]:
df.head()
#df[df.name == "Gloomhaven"]
#df[df.name != "Gloomhaven"]

,Unnamed: 0,Unnamed: 0.1,id,families,categories,mechanics,designers,artists,publishers,description,...,name,usersrated,average,bayesaverage,stddev,median,owned,numweights,averageweight,ranks
0,0,0,174430.0,"['Campaign Games', 'Components: Miniatures', '...","['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Action Retrieval', 'Campaign / Battle Card D...",['Isaac Childres'],"['Alexandr Elichev', 'Josh T. McDowell', 'Alva...","['Cephalofair Games', 'Albi', 'Asmodee', 'Feue...",Gloomhaven is a game of Euro-inspired tactica...,...,Gloomhaven,35170.0,8.82882,8.57357,1.60848,0.0,56525.0,1514.0,3.8276,"[{'name': 'boardgame', 'friendlyname': 'Board ..."
1,1,1,161936.0,"['Campaign Games', 'Legacy', 'Pandemic', 'Them...","['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Rob Daviau', 'Matt Leacock']",['Chris Quilliams'],"['Z-Man Games, Inc.', 'Asterion Press', 'Devir...",Pandemic Legacy is a co-operative campaign gam...,...,Pandemic Legacy: Season 1,37181.0,8.62319,8.47116,1.58066,0.0,58807.0,1039.0,2.8296,"[{'name': 'boardgame', 'friendlyname': 'Board ..."
2,2,2,167791.0,"[""Fryxgames' Future Timeline"", 'Player Count: ...","['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma...",['Jacob Fryxelius'],['Isaac Fryxelius'],"['FryxGames', 'Arclight', 'Fantasmagoria', 'Gh...","In the 2400s, mankind begins to terraform the ...",...,Terraforming Mars,54256.0,8.42645,8.27613,1.37754,0.0,71333.0,2166.0,3.2364,"[{'name': 'boardgame', 'friendlyname': 'Board ..."
3,3,3,224517.0,"['Brass', 'Cities: Birmingham (England)', 'Cou...","['Economic', 'Industry / Manufacturing', 'Tran...","['Connections', 'Hand Management', 'Income', '...","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Lina Cossette', 'David Forest', 'Damien Mamm...","['Roxley', 'BoardM Factory', 'Conclave Editora...",Brass: Birmingham is an economic strategy game...,...,Brass: Birmingham,12937.0,8.63543,8.27466,1.25023,0.0,19441.0,597.0,3.9313,"[{'name': 'boardgame', 'friendlyname': 'Board ..."
4,4,4,182028.0,"['Tableau Building', 'Through the Ages']","['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:...",['Vlaada Chvátil'],"['Filip Murmak', 'Radim Pech', 'Jakub Politzer...","['Czech Games Edition', 'Cranio Creations', 'D...",Through the Ages: A New Story of Civilization ...,...,Through the Ages: A New Story of Civilization,20096.0,8.46737,8.22064,1.50374,0.0,23760.0,991.0,4.3956,"[{'name': 'boardgame', 'friendlyname': 'Board ..."


## Step 3
Perform a correlation analysis of the variables. On top of this, try to design and implement a model for predicting the "subjective" variable values starting from the "objective" variables. Optionally, define a recommendation system for games.

## Step 4
Include in the paper a report describing the output of Step 3